In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
import requests

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from nltk import PorterStemmer, WordNetLemmatizer, pos_tag, word_tokenize, sent_tokenize
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import wordnet
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [152]:
raw_df = pd.read_excel('Hack_Review_oos.xlsx')

##### Combining review summary and review  text

In [140]:
raw_df['conc_review'] = raw_df["Summary"]+" "+raw_df['Review_Text']
raw_df.head()

,Unnamed: 0,Reviewer_ID,Asin,Reviewer_Name,helpful_UpVote,Total_Votes,Review_Text,Rating,Summary,Unix_Review_Time,Review_Time,conc_review,sentiment
0,0,A2CX7LUOHB2NDG,321732944,Bernie,0,0,While many beginner DVDs try to teach you ever...,5,Adobe Photoshop CS5 Crash Course with master P...,1341100800,"07 1, 2012",Adobe Photoshop CS5 Crash Course with master P...,positive
1,1,A1GI0U4ZRJA8WN,439886341,Truthfull,4,4,Do not waste your money on this thing it is te...,1,TERRIBLE DONT WASTE YOUR MONEY,1334707200,"04 18, 2012",TERRIBLE DONT WASTE YOUR MONEY Do not waste yo...,negative
2,2,A29LPQQDG7LD5J,528881469,Alfred Delgado,0,0,I have owned two Rand McNally GPS devices over...,1,Piece of Junk,1352073600,"11 5, 2012",Piece of Junk I have owned two Rand McNally GP...,negative
3,3,AMO214LNFCEI4,528881469,Amazon Customer,12,15,"I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010",Very Disappointed I'm a professional OTR truck...,negative
4,4,A28B1G1MSJ6OO1,528881469,A. O Sandness,21,22,This is a great truck GPS. I've tried others a...,4,Great Unit!,1280016000,"07 25, 2010",Great Unit! This is a great truck GPS. I've tr...,positive


Creating sentiment based on rating

In [67]:
raw_df['sentiment'] = pd.cut(raw_df['Rating'],bins = [0,2,3,5], labels =['negative','neutral','positive'])

In [141]:
raw_df.head(2)

,Unnamed: 0,Reviewer_ID,Asin,Reviewer_Name,helpful_UpVote,Total_Votes,Review_Text,Rating,Summary,Unix_Review_Time,Review_Time,conc_review,sentiment
0,0,A2CX7LUOHB2NDG,321732944,Bernie,0,0,While many beginner DVDs try to teach you ever...,5,Adobe Photoshop CS5 Crash Course with master P...,1341100800,"07 1, 2012",Adobe Photoshop CS5 Crash Course with master P...,positive
1,1,A1GI0U4ZRJA8WN,439886341,Truthfull,4,4,Do not waste your money on this thing it is te...,1,TERRIBLE DONT WASTE YOUR MONEY,1334707200,"04 18, 2012",TERRIBLE DONT WASTE YOUR MONEY Do not waste yo...,negative


### lower case

In [153]:

raw_df['conc_review']=[str(x) for x in raw_df['conc_review']]
raw_df['conc_review']=[x.lower() for x in raw_df['conc_review']]


In [154]:
raw_df.head(2)['conc_review'].iloc[0]
raw_df.head(2)

,ID,conc_review
0,1,"i bought two sets of these, the red/black and ..."
1,2,"i am not very good at packing things, but i ha..."


### Removing stopwords, and punctuation

In [155]:
import string
from nltk import corpus
from nltk.corpus import stopwords
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation

    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
       
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')].string.lower()

In [147]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Grover
[nltk_data]     Anjali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [156]:
raw_df['conc_review'] = [char for char in raw_df['conc_review'] if char not in string.punctuation]
raw_df['conc_review'] = [word for word in raw_df['conc_review'] if word not in stopwords.words('english')]

In [157]:
# !pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
  
# function to print sentiments 
# of the sentence. 
def sentiment_scores(sentence): 

    sid_obj = SentimentIntensityAnalyzer() 

    sentiment_dict = sid_obj.polarity_scores(sentence) 

    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.03 : 
        return("positive") 
  
    elif sentiment_dict['compound'] >= -0.03  : 
        return("neutral") 
  
    else : 
        return("negative") 

### Testing Results 

In [121]:
df_test = raw_df.head(6)

In [158]:
raw_df['Result'] = raw_df['conc_review'].apply(sentiment_scores)

In [161]:
raw_df

,ID,conc_review,Result
0,1,"i bought two sets of these, the red/black and ...",positive
1,2,"i am not very good at packing things, but i ha...",positive
2,3,these are not compression cubes. they are regu...,negative
3,4,cases where not the size they looked on the we...,neutral
4,5,\ni used my packing cubes 3 times and one is s...,negative
5,6,the silicone/rubber sealing ring absorbs food ...,positive
6,7,bought the 6 quart and 3 quart. the 3 quart is...,negative
7,8,instapot advertising seems to primarily focus ...,positive
8,9,"i love this camera, very fast, light weight. j...",positive
9,10,i love this camera. it has features that make ...,positive


In [160]:
raw_df.to_csv('Results.csv')